<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/112_%E6%96%87%E5%AD%97%E5%88%97%E3%81%AE%E6%9B%B8%E5%BC%8F%E6%8C%87%E5%AE%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

文字列の書式指定
================

文字列の内部表現
----------------

Python のテキストデータは `str` 型（文字列）として扱われる。C の `char` のように「1 文字専用の型」は存在しない。

Python の文字列の内部表現は、Unicode コードポイントの並びであり、UTF-8 や UTF-16 といった符号化方式を用いていない。メモリ効率最適化のため、含まれる文字の最大のコードポイントに応じて、以下のいずれかの形式が用いられる（[PEP 393](https://peps.python.org/pep-0393/)）。

  * **1-byte kind**（Latin-1）: 全ての文字が U+00FF 以下の場合。1 文字あたり 1 バイト。
  * **2-byte kind**（UCS-2）: 全ての文字が U+FFFF 以下（基本多言語面、BMP）の場合。1 文字あたり 2 バイト。
  * **4-byte kind**（UCS-4）: U+10000 以上の補助面の文字（絵文字など）を含む場合。1 文字あたり 4 バイト。

CPython の文字列の実装は構造体で、文字列データは `(文字数 + 1) * (kindに応じたバイト数)` の連続したメモリ領域に格納される。`+1` は C との互換性のための終端 NULL 文字の分である（[cpython/Objects/unicodeobject.c](https://github.com/python/cpython/blob/0d7f61ddb074659d8c18c8f5ac86a6a18e41f9e5/Objects/unicodeobject.c#L1447)）。

次のコードは、`sys.getsizeof()` 関数を使用して、文字列のメモリ消費量を比較する例である。`sys.getsizeof()` が返す値は構造体全体のメモリ消費量となるので、差分を取って 1 文字分のメモリ消費量を計算している。

In [ ]:
import sys
x = 'abc'
assert sys.getsizeof(x + 'a') - sys.getsizeof(x) == 1
x = 'abcあ'
assert sys.getsizeof(x + 'a') - sys.getsizeof(x) == 2
x = 'abc𠮷'
assert sys.getsizeof(x + 'a') - sys.getsizeof(x) == 4

組み込み関数 `ord()` は、1 文字の文字列を受け取り、その Unicode コードポイントを表す整数を返す。

組み込み関数 `chr(i)` は、`ord()` の逆の操作であり、整数を受け取り、その値を Unicode コードポイントとする 1 文字の文字列を返す。引数の有効な範囲は 0 から 1,114,111（ 16 進数で 0x10FFFF）。

In [11]:
assert ord('a') == 97       # U+0061
assert ord('あ') == 12354   # U+3042
assert ord('𠮷') == 134071  # U+20BB7
assert chr(97) == 'a'
assert chr(12354) == 'あ'
assert chr(134071) == '𠮷'

Python の文字列は、イミュータブルな Unicode コードポイントのシーケンスとして扱われる。

  * `str` オブジェクトは、生成後に内容を直接書き換えることはできない。
  * `str` オブジェクトは自身の内部表現形式を把握しているため、`len()` 関数は常にバイト数ではなく Unicode コードポイントの数（つまり文字数）を正しく返す。
  * `max()`, `min()` 関数は、それぞれ Unicode コードポイントが最大、最小の文字を返す。

In [6]:
assert len('abc') == 3
assert len( 'abcあ') == 4
assert len('abc😀') == 4
# 結合文字やゼロ幅ジョインナーを使った絵文字は複数のコードポイントを組み合わせているため
# len() の結果は見た目の文字数と異なることに注意！
assert len('é') == 2
assert len('👩‍👩‍👧‍👦') == 7

assert ord("い") == 12356
assert ord("ろ") == 12429
assert ord("は") == 12399
assert max("いろは") == "ろ"
assert min("いろは") == "い"

文字列型への型変換
------------------

組み込み関数 `str()` は文字列型のコンストラクタであり、文字列型への型変換に使用される。

``` python
str(object='')
```

`str()` は、単一の引数を与えられ場合、内部的に第 1 引数に与えられたオブジェクトの `__str__()` メソッドを呼び出し、そのオブジェクトの「人間が読みやすい文字列表現」を返す。文字列オブジェクトに対しては文字列そのものを返し、バイト列オブジェクトに対してはそのリテラル表現の文字列を返す。オブジェクトが `__str__()` メソッドを持たない場合、 `str()` は代わりに `repr(object)` の結果を返す。引数が指定されない場合は空文字列を返す。

In [1]:
assert str(10) == '10'
assert str(-1.23) == '-1.23'
assert str(1.23+45j) == '(1.23+45j)'
assert str() == ''

整数と文字列の変換は `sys.int_info.default_max_str_digits` 桁（デフォルトで 4300 桁）までに制限されることに注意。

In [ ]:
try:
    str(10**4300)  # 4301 桁の整数から文字列への型変換
except Exception as e:
    print(f"{type(e).__name__}: {e}")

ValueError: Exceeds the limit (4300 digits) for integer string conversion; use sys.set_int_max_str_digits() to increase the limit


正の無限大を文字列に変換すると `'inf'` が得られる。負の無限大を文字列に変換すると `'-inf'` が得られる。

In [ ]:
import math
assert str(float('Infinity')) == str(math.inf) == 'inf'
assert str(-float('Infinity')) == str(-math.inf) == '-inf'

NaN を文字列に変換すると `'nan'` が得られる。

In [ ]:
assert str(float('NaN')) == 'nan'

組み込み関数 `bin()` は、引数として渡された整数を、先頭に `'0b'` が付いた 2 進数表現の文字列に変換する。負の数を渡した場合に、2 の補数形式ではなく絶対値にマイナス符号が付いた形で返される。

In [ ]:
assert bin(4) == '0b100'
assert bin(-4) == '-0b100'  # 0b1100 ではない

組み込み関数 `oct()` は、引数として渡された整数を、先頭に `'0o'` が付いた 8 進数表現の文字列に変換する。

In [ ]:
assert oct(8) == '0o10'
assert oct(-56) == '-0o70'

組み込み関数 `hex()` は、引数として渡された整数を、先頭に `'0x'` が付いた小文字の 16 進数表現の文字列に変換する。

In [ ]:
assert hex(255) == '0xff'
assert hex(-42) == '-0x2a'

文字列リテラル
--------------

次の構文による**文字列リテラル**は、文字列を生成する。

``` python
[<stringprefix>]<引用符><文字列><引用符>
```

  * stringprefix （任意）  
stringprefix と引用符の間に空白があってはならない。stringprefix は大文字小文字の区別がない。

| stringprefix | 意味 |
|:---|:---|
| `r` | raw 文字列リテラル（raw-string） |
| `f` | フォーマット済み文字列リテラル（f-string） |
| `fr` または `rf` | フォーマット済みの raw 文字列リテラル |

  * 引用符 （必須）  
一重引用符・二重引用符・三重引用符（3 つの一重引用符か 3 つの二重引用符）が使える。三重引用符文字列は、複数行に分けることができる。
  * 文字列 （必須）  
raw 文字列リテラルでない限り、文字列中の `'\'` と文字の組み合わせでエスケープシーケンスと解釈される:

| ｴｽｹｰﾌﾟｼｰｹﾝｽ | 意味 |
|:--:|:---|
| `\改行` | バックスラッシュと改行文字が無視される |
| `\\` | バックスラッシュ (`\`) |
| `\'` | 一重引用符 (`'`) |
| `\"` | 二重引用符 (`"`) |
| `\a` | ASCII 端末ベル (BEL) |
| `\b` | ASCII バックスペース (BS) |
| `\n` | 改行 (LF) |
| `\f` | ASCII フォームフィード (FF) |
| `\r` | ASCII 復帰 (CR) |
| `\t` | ASCII 水平タブ (TAB) |
| `\v` | ASCII 垂直タブ (VT) |
| `\ooo` | 8 進数値 ooo をコードポイントとする文字（ooo は最大で 3 桁） |
| `\xhh` | 2 桁の 16 進数値 hh をコードポイントとする文字 |
| `\uxxxx` | 4 桁の 16 進数値 xxxx をコードポイントとする文字 |
| `\Uxxxxxxxx` | 8 桁 の16 進数値 xxxxxxxx をコードポイントとする文字 |
| `\N{name}` | Unicode データベース中で name という名前の文字 |

一重引用符文字列または二重引用符文字列は、単一の式の一部であり間に空白のみを含むなら、一つの文字列リテラルに暗黙に変換される。つまり、`("spam " "eggs") == "spam eggs"` が成立する。このとき、一重引用符文字列と二重引用符文字列が混在していてもよい。`()` で囲むと改行を入れることもできる。

In [ ]:
assert "XY" '\u005a' == "XYZ"
text = ("Beautiful"
        " is"
        " better"
        " than"
        " ugly.")
assert text == "Beautiful is better than ugly."

なお、組み込み関数 `ascii(object)` は、`repr(object)` を呼び出し、それによって返された文字列中の非 ASCII 文字を `\x`、 `\u`、`\U` エスケープを使ってエスケープした文字列を返す。

In [15]:
assert ascii("こんにちは") == "'\\u3053\\u3093\\u306b\\u3061\\u306f'"

書式指定
--------

### 書式化に関するプロトコル

f-string や `str.format()` メソッドを適用する文字列には、波括弧 `{}` で区切られた式である**置換フィールド**（replacement field）を含めることができる。その文字列は、実行時に次のように評価される。

  * 置換フィールド以外は文字通り解釈される。
  * 置換フィールドは共通の書式化に関するプロトコルに沿って評価された結果に置き換えられる。

ただし、二重波括弧 `'{{'` および `'}}'` は単一の波括弧に置き換えられる。つまり、波括弧を文字として扱う必要がある場合は、二重にすることでエスケープすることができる。

In [ ]:
x = "hoge"
assert f"{{{x}}}" == "{hoge}"

書式化に関するプロトコルは、置換フィールドのオブジェクトが次の特殊メソッドをサポートすることを要求する。

  * `__format__(format_spec)`

`__format__()` メソッドは、オプションを渡すことができる文字列フォーマットのための `__str__()` と考えることができる。組み込み関数 `format()` は、`format_spec` のデフォルトを空の文字列として、対象のオブジェクトの `__format__()` メソッドに全ての処理を丸投げする。実際、`format()` の実装を Python で書くと、次のようになる（PEP 3101 より）:

``` python
def format(value, format_spec=''):
    return value.__format__(format_spec)
```

オブジェクトのクラスが `__format__()` を定義していない場合は、`object` から継承される `object.__format__()` メソッドが呼び出される。このメソッドの中からは、対象のオブジェクトの `__str__()` メソッドが呼び出される。`object.__format__()` の定義を Python で書くと、次のようになる（PEP 3101 より）:

``` python
class object:
    def __format__(self, format_spec):
        return format(str(self), format_spec)
```

オブジェクトのクラスが `__str__()` も定義していない場合は、`object` から継承される `object.__str__()` メソッドが呼び出される。`object.__str__()` は `__repr__()` メソッドを呼び出すだけである。

置換フィールドでは、コロン `:` の後ろが `format_spec` の指定となる。置換フィールドに対して行われる操作は、実のところ、オブジェクト `value` と `format_spec` を引数として渡して `format(value, format_spec)` を呼び出しているだけである。

書式化に関するプロトコルによって f-string や `str.format()` メソッドは任意の型（ユーザー定義型であっても）をサポートするようになっている。次のコードでは、`Country` クラスに `__format__()` メソッドを定義している。

In [ ]:
class Country:
    def __init__(self, name, iso):
        self.name = name
        self.iso = iso

    def __format__(self, format_spec):
        if format_spec == "short":
            return self.iso
        return self.name


if __name__ == "__main__":
    jp = Country("Japan", "JPN")
    assert f"{jp}" == "Japan"
    assert f"{jp:short}" == "JPN"

### 置換フィールドの構文

f-string では、置換フィールド `{...}` の構文は次のようになる。

``` python
{<f_expression>[=][!<conversion>][:<format_spec>]}
```

  * f_expression （必須）  
ラムダ式以外の式なら何でも書ける。
  * `=` （任意）  
f_expression の評価結果を `<f_expression>=<value>` の形式の文字列に変換する。
  * conversion （任意）  
指定されていた場合、f_expression の評価結果は `format()` 呼び出しの前に変換される。事実上、`__format__()` をオーバーライドする形となる。

| conversion | 意味 | 備考 |
|:--:|:---|:---|
| `s` | `str()` を呼び出す | `=` を指定し、書式指定が存在する場合のデフォルト |
| `r` | `repr() ` を呼び出す | `=` を指定し、書式指定が存在しない場合のデフォルト |
| `a` | `ascii()` を呼び出す | |

  * format_spec （任意）  
書式指定。format_spec の形式・意味は、`type(value).__format__()` の実装に依存する。つまり、`value` の型ごとに format_spec の形式・意味は異なる。このため、形式上同じ format_spec であっても、型によって解釈が異なることがある。

In [ ]:
text = "python"
assert f"{text=:0}" == "text=python"  # デフォルトでは str(text) が呼び出される
assert f"{text=}" == "text='python'"  # デフォルトでは repr(text) が呼び出される
assert f"{text=!r:0}" == "text='python'"

フォーマット済み文字列リテラルは、たとえ式を含んでいなかったとしても、docstring としては使えない。

In [ ]:
def foo():
    f"Not a docstring"

assert foo.__doc__ is None

Python 3.12 からは、置換フィールドの構文が以下のように拡張された。

① f-string 全体の引用符と同じ文字を、式の中で使える。

In [16]:
f"{f"{f"{"f文字列ネストし放題！"}"}"}"

'f文字列ネストし放題！'

② 式の中で `\` や改行が使える。

In [17]:
f"{'\n'}"

'\n'

In [18]:
f"{
1+1
}"

'2'

③ 式の中でコメントが使える。

In [19]:
f"{
1+1  # 1行目
+2+2 # 2行目
}"

'6'

### 文字列に対する書式指定

`str` 型オブジェクトの値に対する書式指定は次のようになる。

``` python
[[<fill>]<align>][0][<width>]
```

  * align （任意）  
配置を指定する。配置により余った部分は fill で指定した文字で埋める。fill が指定されてなければ、空白文字で埋める。利用可能なスペースがデータを表示するために必要な幅と同じなら、align の指定は無効になる。

| align | 意味 |
|:--:|:---|
| `<` | 左詰め（デフォルト） |
| `>` | 右詰め |
| `^` | 中央寄せ |

  * `0` （任意）  
配置により余った部分を `0` で埋める。
  * width （任意）  
非負の 10 進整数で利用可能なスペースを指定する。データを表示するために必要な幅より小さいときは無視される。

`0` 埋めの場合は、fill で指定する方法と、width の前に `0` を付ける方法の二通りあることになる。

In [ ]:
text = "python"
assert f"|{text:<10}|" == "|python    |"
assert f"{text:0>10}"  == f"{text:>010}"  == "0000python"
assert f"{text:-^10}"  == "--python--"

### 整数に対する書式指定

`int` 型オブジェクトの値に対する書式指定は次のようになる。

``` python
[[<fill>]<align>][<sign>][#][0][<width>][<grouping_option>][<type>]
```

  * align （任意）  
意味は一般の文字列の場合と同じ。デフォルトと `=` が違う。

| align | 意味 |
|:--:|:---|
| `<` | 左詰め |
| `>` | 右詰め（デフォルト） |
| `^` | 中央寄せ |
| `=` | 符号があれば、その後ろを埋める（`0=` は `0<width>` と同じ） |

  * sign （任意）  
符号の使用を強制する。実際に意味があるのは、正の数値に対して `+` を指定するときだけ。
  * `#` （任意）  
2 進法、8 進法、または 16 進法の出力が使用される場合、このオプションは出力される値にそれぞれ `0b`, `0o`, `0x`, `0X` 接頭辞を加える。
  * `0` （任意）  
align が与えられていないときに、`0<width>` は `0=` の align が与えられた場合と同じである。
  * width （任意）  
意味は一般の文字列の場合と同じ。符号も幅にカウントされる。
  * grouping_option （任意）  
千の位のセパレーター文字を付ける。`_` か `,` を指定する。
  * type （任意）  
整数の表現形式を指定する。

| type | 意味 |
|:--:|:---|
| `d` | 10 進数（デフォルト） |
| `b` | 2 進数 |
| `o` | 8 進数 |
| `x` | 16進数（10 進で 9 を超える数字には小文字が使われる） |
| `X` | 16進数（10 進で 9 を超える数字には大文字が使われる） |
| `c` | 数値に対応する Unicode 文字 |
| `n` | 現在のロケールに従い、区切り文字を挿入することを除けば、`d` と同じ |
| `None` | `d` と同じ |

In [ ]:
value = 1000
assert f"{-value:*=10}" == "-*****1000"
assert f"{value:+010}" == "+000001000"
assert f"{value:,}" == "1,000"
assert f"{value:b} {value:o} {value:x} {value:X}" == "1111101000 1750 3e8 3E8"
assert f"{value:#b} {value:#o} {value:#x} {value:#X}" == "0b1111101000 0o1750 0x3e8 0X3E8"
assert f"{value:c}" == "Ϩ"  # コードポイント U+03E8 の文字

### 浮動小数点数に対する書式指定

`float` 型オブジェクトの値に対する書式指定は以下のようになる。

``` python
[[<fill>]<align>][<sign>][z][#][0][<width>][<grouping_option>][.<precision>][<type>]
```

  * align （任意）  
意味は整数の場合と同じ。
  * sign （任意）  
意味は整数の場合と同じ。
  * `z` （任意）[Python 3.11 で追加]  
負の値を丸め処理したことによりゼロ値になったとき、符号 `-` を外す。たとえば、`-0.0` なら `0.0` とする。
  * `#` （任意）  
小数点文字の後に数字がない場合でも、このオプションは小数点文字を加える。
  * `0` （任意）  
意味は整数の場合と同じ。
  * width （任意）  
意味は整数の場合と同じ。符号と小数点文字も幅にカウントされる。
  * grouping_option （任意）  
意味は整数の場合と同じ。
  * precision （任意）  
固定小数点数表記をする場合に、小数点以下の桁数（精度）を非負の 10 進整数で指定する。デフォルトは小数点以下 6 桁になる。精度が不足するときは `round()` 関数と同じ丸め処理を行う。
  * type （任意）  
浮動小数点数の表現形式を指定する。

| type | 意味 |
|:--:|:---|
| `e` | 指数表記。小文字の `e` を使う |
| `E` | 指数表記。大文字の `E` を使う |
| `f` | 固定小数点数表記 |
| `F` | 固定小数点数表記。nan が NAN に、inf が INF に変換されることを除き `f` と同じ |
| `g` | 汎用フォーマット。数値が大きくなったとき、`e` に切り替わる（デフォルト） |
| `G` | 汎用フォーマット。数値が大きくなったとき、`E` に切り替わる。無限大と NaN の表示も大文字 |
| `n` | 現在のロケールに合わせて、数値分割文字が挿入されることを除き、`g` と同じ |
| `%` | 百分率表記 |
| `None` | `g` と同じ |

また、整数に対して、`n` と `None` を除く浮動小数点数の表現形式を指定すると、`float()` を使って浮動小数点数に変換したうえで書式変換される。

In [ ]:
assert f"{12.345:f}" == "12.345000"
assert f"{-12.345:8.2f}" == "  -12.35"
assert f"{12.345:e}" == "1.234500e+01"
assert f"{0.0045:.2%}" == "0.45%"
assert f"{100:f}" == "100.000000"

### str.format()

`str.format(*args, **kwargs)` メソッドでは、置換フィールド内にオブジェクトを書くのではなく、引数にオブジェクトを指定する。置換フィールド内は全て文字列として扱われる。置換フィールドと引数の対応づけは、次の 3 通りある。

  * 置換フィールド内に番号を書く場合: その番号と位置引数の順番で対応付けが行われる。
  * 置換フィールド内にキーワードを書く場合: キーワード引数で対応付けが行われる。
  * 置換フィールド内に対応付けを書かない場合: 置換フィールドが現れる順番と位置引数の順番で対応付けが行われる（0, 1, 2, ... の番号が自動的にその順で挿入される）。

また、置換フィールド内で `!` の後ろに conversion を書くことができ、`:` の後ろに format_spec（書式指定）を書くことができる（`=` は書けない）。それぞれの役割は f-string と共通である。

In [ ]:
assert "{2}, {0:06}, {1:#o}".format(64, 128, 256) == "256, 000064, 0o200"
assert "{a}, {b:06}, {c:#o}".format(b=64, c=128, a=256) == "256, 000064, 0o200"
assert "{}, {:06}, {:#o}".format(64, 128, 256) == "64, 000128, 0o400"

また、 `str.format()` の置換フィールドでは、番号やキーワードの後ろに添字表記 `[]` やドット `.` 属性形式を続けることができる。これらについては、それぞれ `__getitem__()` や `getattr()` を使用してインデックス参照や属性参照を行う。ただし

  * リストの場合、 `[]` の中でコロン `:` を使ったスライス表記や負の整数を使ってはならない。
  * 辞書の場合、`[]` の中でキーの指定は直接キー名を使うことに限られ、キー名を引用符（`'`, `"`）で囲ってはならない。

という制限があることに注意する。もっと自由にインデックス参照や属性参照を行いたい場合は、置換フィールド内に式が使える f-string を使う必要がある。

In [ ]:
li = ["foo", "baa", "baz"]
dic = {"Japan": "Tokyo", "USA": "Washington, D.C."}
print("{0[0]} -- {1[Japan]} -- {0[1]} --- {1[USA]}".format(li, dic))

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

print("Point: ({p.x}, {p.y})".format(p=Point(4, 3)))

foo -- Tokyo -- baa --- Washington, D.C.
Point: (4, 3)


f-string と `str.format()` メソッドのどちらが優れているとは一概には言えない。

f-string のメリット:

  * 直観的に文字列の書式化を規定できる。
  * 置換フィールドと引数の対応付けに関するコーディングミスという問題を解決する。
  * 置換フィールドに任意の式を書くことができる。

f-string のデメリット:

  * f-string は直ちにフォーマット済み文字列を組み立てるので、文字列の遅延評価ができない。
  * 置換フィールドに長い式を書く場合、フォーマット済み文字列の可読性が下がる。ただし、長い式の代わりにヘルパー関数を書くという解決策はある。

f-string のメリットとデメリットのそれぞれの否定が、ちょうど `str.format()` メソッドのデメリット・メリットになる。とくに、`str.format()` を使うと、文字列の遅延評価、すなわち、置換フィールドを含む文字列だけ用意して、必要になった段階でフォーマット化するということができる。実行時にフォーマット済み文字列がいつ必要になるかわからない場合は、文字列の遅延評価を行えばフォーマット化が無駄にならない。

メリット・デメリットを考慮して、ケースバイケースで f-string と `str.format()` メソッドを使い分けることでよい。

### % 形式の文字列書式化

`%` 演算子を用いて文字列書式化が可能である。

``` python
format % values
```

このように書くと、文字列 `format` の中の % 変換指定が `values` の要素で置換される。これは、C の sprintf() 関数と同等で、書式化に関するプロトコルが導入される前の古い形式である。

% 変換指定の構文は次のとおり。

``` python
%[(<mapkey>)][<flag>][<width>][.<precision>]<specifier>
```

マップキー（mapkey）は、丸括弧で囲った文字列からなる。マップキーを指定する場合、`values` は辞書でなければならない。マップキーを指定しない場合は、% 変換指定が 1 個なら `values` は単一のオブジェクトとし、% 変換指定が複数個なら `values` は同数の要素を持つタプルとしなければならない。

オプションの変換フラグ（flag）を以下に示す:

| Flag | 機能 |
|:---|:---|
| `'#'` | 一部の変換型に対して別の形式を使う |
| `'0'` | 数値型に対してゼロによるパディングを行う |
| `'-'` | 変換された値を左寄せにする（`'0'` フラグと同時に与えた場合、`'0'` フラグを上書きする） |
| `' '` | 符号付きの変換で正の数の場合、前に 1 つスペースを空ける（そうでない場合は空文字になる） |
| `'+'` | 変換の先頭に符号文字（`'+'` または `'-'`) を付ける（`' '` フラグを上書きする） |

オプションの幅（width）を指定した場合は、その指定した幅と実際の文字列幅のうち大きいほうが採用される。

オプションの精度（precision）は、ドット `'.'` とその後に続く精度（小数点以下の桁数）で与えられる。

変換型（specifier）の指定は必須で、どのような型として置換するかを、以下の文字で指定する:

| 変換 | 意味 |
|:--:|:---|
| `'d'` | 符号付き 10 進整数 |
| `'i'` | 符号付き 10 進整数 |
| `'o'` | 符号付き 8 進整数。変換フラグ `'#'` を使用すると接頭辞 `'0o'` が挿入される |
| `'x'` | 符号付き 16 進整数（小文字）。変換フラグ `'#'` を使用すると接頭辞 `'0x'` が挿入される |
| `'X'` | 符号付き 16 進整数（大文字）。変換フラグ `'#'` を使用すると接頭辞 `'0X'` が挿入される |
| `'e'` | 指数表記の浮動小数点数（小文字） |
| `'E'` | 指数表記の浮動小数点数（大文字） |
| `'f'` または `'F'` | 10 進固定小数点数表記の浮動小数点数 |
| `'c'` | 文字一文字（整数または一文字からなる文字列を受理する） |
| `'r'` | 文字列（Python オブジェクトを `repr()` で変換する）。精度が N なら、出力は（小数点 `.` を含めて） N 文字に切り詰められる |
| `'s'` | 文字列（Python オブジェクトを `str()` で変換する）。精度が N なら、出力は（小数点 `.` を含めて） N 文字に切り詰められる |
| `'a'` | 文字列（Python オブジェクトを `ascii()` で変換する）。精度が N なら、出力は（小数点 `.` を含めて） N 文字に切り詰められる |

また、`format` の中で文字 `'%'` を表したいときは、`'%%'` と書く。

In [ ]:
# マップキーなし
assert "%%%s%%" % "hoge" == "%hoge%"
assert "|%-10f|%10f|% .3f|%x|%#x|" % (1.23, 1.23, 1.23, 123, 123) == "|1.230000  |  1.230000| 1.230|7b|0x7b|"

# マップキーあり
assert "%(month)02d月%(day)02d日%(one)s曜日" % dict(month=10, day=8, one="日") == "10月08日日曜日"

% 形式の文字列書式化では、オブジェクトごとに特別な書式化を定義できない。この形式は、使われる場面が減っているが、文字列の遅延評価が可能であること、および、`str.format()` メソッドより速いこと（※）から、ログ出力の書式化に使われている。

※ [pythonのf-stringとformatとパーセント%の書式の速度の比較](https://qiita.com/phyblas/items/0a122fe4a18a18a83050)

ロケール
--------

ロケールとは、国や地域の文化によって異なる言語や単位、表記などの総称である。Python では、標準ライブラリの `locale` モジュールによってロケールの機能を利用できる。`locale` は、POSIX 規格に従って、ロケールを用途別に分類し、以下のカテゴリー定数を定義している。

| カテゴリー定数 | 意味 |
|:--|:--|
| `locale.LC_CTYPE` | 文字の分類（文字とはどんなもの？大文字小文字を区別しない？） |
| `locale.LC_NUMERIC` | 数字の書式（小数区切り文字、千の区切り文字など） |
| `locale.LC_TIME` | 日付と時刻の書式 |
| `locale.LC_COLLATE` | 文字列の並び換え順 |
| `locale.LC_MONETARY` | 通貨書式（通貨記号、千の区切り文字など） |

ロケールの名前は、POSIX 規格のものが使える。POSIX 互換システムなら、コマンド `locale -a` で使用可能なロケールが出力される。多くのシステムでは、以下のものが使える（それぞれ `<言語>_<地域>.<文字コードセット>` という命名規則に従っている）。

| ロケール | 説明 |
|:--|:--|
| `'de_DE.UTF-8'` | ドイツ語、ドイツ |
| `'en_US.UTF-8'` | 英語、アメリカ |
| `'es_ES.UTF-8'` | スペイン語、スペイン |
| `'fr_FR.UTF-8'` | フランス語、フランス |
| `'it_IT.UTF-8'` | イタリア語、イタリア |
| `'ja_JP.UTF-8'` | 日本語、日本 |
| `'ko_KR.UTF-8'` | 韓国語、韓国 |
| `'pt_BR.UTF-8'` | ポルトガル語、ブラジル |
| `'zh_CN.UTF-8'` | 中国語（簡体字）、中国 |
| `'zh_TW.UTF-8'` | 中国語（繁体字）、台湾 |

これらのうち、Colab のシステムで使えるロケールは `'en_US.UTF-8'` だけであることに注意する。

In [20]:
!locale -a

C
C.utf8
en_US.utf8
POSIX


``` python
locale.getlocale(category=LC_CTYPE)
```

この関数は、カテゴリー定数で指定する `category` に関して現在使用しているロケールを、言語コードと、エンコーディングを含むシーケンスで返す。言語コードおよびエンコーディングが決定できなかった場合は、`None` になる。

``` python
locale.setlocale(category, locale=None)
```

この関数は、カテゴリー定数で指定する `category` に関して使用するロケールを `locale` に変更する。`category` に `locale.LC_ALL` を指定すると、すべてのカテゴリーのロケールを変更する。`locale` に `'C'` を設定するとロケール機能を無効化する。システムにインストールされていないロケールを指定すると、エラーになる。

``` python
>>> import locale
>>> locale.getlocale(locale.LC_NUMERIC)
(None, None)
>>> val = 12345.6
>>> f"{val:n}"
'12345.6'
>>> locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
'en_US.UTF-8'
>>> f"{val:n}"
'12,345.6'
>>> locale.setlocale(locale.LC_ALL, "de_DE.UTF-8")
'de_DE.UTF-8'
>>> f"{val:n}"
'12.345,6'
>>>
```